# **🧠 O que é o ciclo BDI?**

> ⚠️ Este Colab não foi feito para execução do codigo, criei por motivos de explicabilidade do codigo. O codigo deve ser rodado localmente no caminho /code.

O **BDI (Belief-Desire-Intention)** é uma arquitetura de agentes baseada na forma como humanos tomam decisões racionais.

O ciclo BDI funciona em três camadas:

*   **Beliefs (Crenças): o que o agente sabe sobre o mundo.**
*   **Desires (Desejos): o que ele quer alcançar.**
*   **Intentions (Intenções): os planos escolhidos para agir com base nas crenças e desejos.**


O agente repete continuamente esse ciclo:

1. **Atualiza suas crenças com novas informações (ex: uma erva daninha detectada).**

2. **Seleciona desejos relevantes com base nesse estado.**

3. **Escolhe planos válidos (cujos contextos estão de acordo com as crenças).**

4. **Executa ações para alcançar os objetivos.**

5. **Volta ao início e reavalia tudo.**








In [ ]:
import requests
import random
import time
import threading
from flask import Flask, request, jsonify

### **Agente Supervisor** — `agent_supervisor.py`
- O agente supervisor atua como um hardware mais avançado para detecção das anomalias nas plantas, usando inteligencia artificial para Analisa imagens recebidas do campo.
- Simula uso de YOLO + Label Studio.
- Detecta ervas daninhas com base em confiança (>0.8).
- Atualiza suas crenças e envia comandos para os demais agentes.
- Comunicação via `/agent` (GET/POST).

> A ***PlanLibrary*** armazena os **planos do agente**, associando cada **objetivo (goal)** a um **contexto necessário (pré-condições)** e a uma **sequência de açõe**s que devem ser executadas quando aquele objetivo for ativado.
A função ***get_plan()*** verifica se o contexto de um plano é compatível com **as crenças atuais (beliefs)**. Se for, retorna o plano associado; caso contrário, ignora.

In [ ]:
class PlanLibrary:
    def __init__(self):
        self.plans = {}

    def set_plan_library(self, planlb):
        self.plans = planlb

    def add_plan(self, goal, prec, plan):
        self.plans[goal] = {'context': prec, 'plan': plan}

    def get_plan(self, goal, bb):
        if goal in self.plans:
            if set(self.plans[goal]['context'].items()).issubset(bb.items()):
                return self.plans[goal]['plan']
        return None

## 🔍 `class Action`

Esta classe define as ações executadas pelo agente supervisor no sistema multiagente BDI. Ele atua como o cérebro analítico da operação, analisando imagens, detectando ameaças e coordenando os agentes de campo e herbicida.

### 🧪 `verificar_erva_daninha()`

Responsável por simular a análise de imagens utilizando um sistema de detecção como YOLO. (no **colab labelstudio_yollo** explico como implementar uma detecção usando o **esp32cam** juntamente com o **opencv** para o carregamento das imagens o **labelstudio** para treinamento personalizado da inteligencia artificial e o **yollov5** como analisador inteligente)

**Fluxo:**
1. Consulta o endpoint `/stream` do agente de campo para obter uma imagem.
2. Simula o processamento da imagem com `sleep()` para representar o tempo de análise do modelo YOLO.
3. Analisa uma lista de objetos simulados, incluindo `"trigo"` e `"erva_daninha"`, com nível de confiança.
4. Se houver erva daninha com confiança > 0.8, atualiza a crença: erva_daninha_detectada: True

5. Se houver erva daninha:
   - Notifica o agente de campo para parar e enviar suas coordenadas.
   - Aciona o agente herbicida para contenção de pragas.

6. Atualiza suas crenças com base nas comunicações realizadas.

In [ ]:
class Action:
    def verificar_erva_daninha(self):
        try:
            print("🎥 Buscando frame do agente de campo...")
            res = requests.get("http://127.0.0.1:5002/stream", timeout=1)
            if res.status_code != 200:
                print("❌ Não foi possível obter o frame.")
                supervisor.add_beliefs({"erva_daninha_detectada": False})
                return

            frame = res.json().get("frame")
            print(f"🧠 Frame recebido: {frame}")
            print(f"🧠 Frame recebido: {frame}")
            print(f"🧠 Frame recebido: {frame}")
            print(f"🧠 Frame recebido: {frame}")

        except Exception as e:
            print(f"💥 Erro ao buscar frame: {e}")
            supervisor.add_beliefs({"erva_daninha_detectada": False})
            return

        print("🔍 YOLO simulado analise...")
        time.sleep(5)
        print("🔍 YOLO simulado analise...")
        time.sleep(5)
        print("🔍 YOLO simulado analise...")
        time.sleep(5)


        detections = [
            {"class": "trigo", "confidence": 0.88},
            {"class": "erva_daninha", "confidence": 0.91},
        ]

        ervas = [d for d in detections if d["class"] == "erva_daninha" and d["confidence"] > 0.8]
        detectado = len(ervas) > 0

        supervisor.add_beliefs({"erva_daninha_detectada": detectado})
        print(f"📦 Simulação estruturada: {'ERVA DANINHA DETECTADA' if detectado else 'Somente trigo'}")
        print(f"📦 Simulação estruturada: {'ERVA DANINHA DETECTADA' if detectado else 'Somente trigo'}")
        print(f"📦 Simulação estruturada: {'ERVA DANINHA DETECTADA' if detectado else 'Somente trigo'}")
        print(f"📦 Simulação estruturada: {'ERVA DANINHA DETECTADA' if detectado else 'Somente trigo'}")

    def enviar_alarme_para_agente_campo(self):
        crença_agente_campo = {"erva_daninha": True}
        try:
            res = requests.post("http://192.168.15.4:5002/agent", json=crença_agente_campo)
            print(f"📡 Enviado ao agente de campo: {crença_agente_campo} | Status: {res.status_code}")
        except Exception as e:
            print(f"❌ Erro ao enviar crença: {e}")

        supervisor.add_beliefs({"agente_campo_comunicado": True})

    def enviar_alarme_para_agente_herbicida(self):
        print("💨 Preparando envio de alarme para o agente herbicida...")
        time.sleep(5)

        payload = {
            "aplicar_herbicida": True,
            "coordenadas_agente_campo": True,
            "erva_daninha_detectada": True,
        }

        try:
            res = requests.post("http://127.0.0.1:5004/agent", json=payload)
            print(f"🧪 Herbicida acionado com sucesso! Status: {res.status_code}")
        except Exception as e:
            print(f"❌ Erro ao contatar agente herbicida: {e}")

        supervisor.add_beliefs({"agente_herbicida_comunicado": True})






### **🤖 Classe Agent (BDI)**

A classe `Agent` implementa a lógica principal de um agente inteligente baseado no modelo BDI (*Beliefs-Desires-Intentions*).

In [ ]:
class Agent:
    def __init__(self):
        self.beliefs = {}
        self.desires = []
        self.intention = []
        self.plan_library = PlanLibrary()

    def get_desires(self):
        return self.desires.pop() if self.desires else None

    def add_desires(self, desire):
        self.desires.append(desire)

    def add_beliefs(self, beliefs):
        self.beliefs.update(beliefs)

    def set_plan_library(self, pl):
        self.plan_library.set_plan_library(pl)

    def update_intention(self, goal):
        plan = self.plan_library.get_plan(goal, self.beliefs)
        if plan:
            self.intention.extend(plan)

    def execute_intention(self):
        while self.intention:
            next_goal = self.intention.pop()
            if self.plan_library.get_plan(next_goal, self.beliefs) is None:
                action = getattr(Action(), next_goal)
                action()
            else:
                self.intention.extend(self.plan_library.get_plan(next_goal, self.beliefs))




### 🗺️ **Planos Definidos (set_plan_library)**

O supervisor possui três planos principais, que são ativados conforme o estado das crenças:

- **Analisar Monitoramento:**  
  Se ainda não foi detectada nenhuma erva daninha, o supervisor executa a ação de verificação de imagem, simulando a análise de um novo frame.

- **Acionar Agente de Campo:**  
  Quando uma erva daninha é detectada e o agente de campo ainda não foi comunicado, o supervisor envia um alerta ao campo, solicitando coordenadas da área afetada.

- **Acionar Agente Herbicida:**  
  Após a detecção e comunicação com o campo, se o herbicida ainda não tiver sido acionado e as coordenadas estiverem disponíveis, o supervisor inicia o processo de contenção acionando o agente herbicida.


🌐 Endpoint `/agent`

O supervisor também expõe um endpoint REST chamado `/agent`, com dois modos:

- `GET`: permite que qualquer outro agente ou usuário consulte as crenças atuais do supervisor.
- `POST`: permite que outros agentes atualizem o estado do supervisor enviando novas crenças.

Esse mecanismo permite que o supervisor se mantenha sincronizado com os demais agentes do sistema e reaja dinamicamente ao ambiente.

In [ ]:
supervisor = Agent()

supervisor.add_beliefs({
    "erva_daninha_detectada": False,
    "agente_campo_comunicado": False,
    "agente_herbicida_comunicado": False,
    "coordenadas_agente_campo": False
})

supervisor.set_plan_library({
    "analisar_monitoramento": {
        "context": {"erva_daninha_detectada": False},
        "plan": ["verificar_erva_daninha"]
    },
    "acionar_agente_campo": {
        "context": {"erva_daninha_detectada": True, "agente_campo_comunicado": False},
        "plan": ["enviar_alarme_para_agente_campo"]
    },
    "acionar_herbicida_central": {
        "context": {"erva_daninha_detectada": True, "agente_herbicida_comunicado": False, "coordenadas_agente_campo": True},
        "plan": ["enviar_alarme_para_agente_herbicida"]
    }
})

app = Flask(__name__)

@app.route("/agent", methods=["GET", "POST"])
def endpoint_agent():
    if request.method == "GET":
        return jsonify(supervisor.beliefs)
    if request.method == "POST":
        novas = request.json
        supervisor.add_beliefs(novas)
        return jsonify({"msg": "Crenças atualizadas", "crencas": supervisor.beliefs})



### **🔁 Ciclo BDI Contínuo (ciclo_agente)**

Este é o **loop principal** que mantém o agente ativo e deliberando de forma autônoma.

In [ ]:
def ciclo_supervisor():
    while True:
        supervisor.add_desires("analisar_monitoramento")
        supervisor.add_desires("acionar_agente_campo")
        supervisor.add_desires("acionar_herbicida_central")

        while True:
            goal = supervisor.get_desires()
            if not goal:
                break
            supervisor.update_intention(goal)
            supervisor.execute_intention()

        time.sleep(5)

if __name__ == "__main__":
    threading.Thread(target=ciclo_supervisor, daemon=True).start()
    print("🧠 Supervisor rodando")
    app.run(host="0.0.0.0", port=5003)